<a href="https://colab.research.google.com/github/protocol-streams/querent-experimental/blob/main/Co_reference_Resolution_Allennlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy==3.1.3
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached thinc-8.0.17-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (659 kB)
  Using cached wasabi-0.10.1-py3-none-any.whl (26 kB)
  Using cached pydantic-1.8.2-py3-none-any.whl (126 kB)
  Created wheel for spacy: filename=spacy-3.1.3-cp310-cp310-linux_x86_64.whl size=26107542 sha256=33144b864045985344f8ac23a406bddf5ff50d5071358c7381b828533c6deeb8
  Stored in directory: /root/.cache/pip/wheels/ab/2d/e6/1d7ba18fa52c1407024a67c34a678d3a67f5784098bb240aac
Successfully built spacy
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling

In [2]:
!pip install allennlp==2.10.1 allennlp-models==2.10.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 819.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 23.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB

https://github.com/allenai/allennlp-models

In [3]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.coref

# Load the coreference resolution model
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Output()

Downloading:   0%|          | 0.00/414 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/634M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Coreference resolution with Allen Institute
What we get as a result (prediction) is a dictionary as Allen outputs multiple different information at once.
The ones that we found to be using the most are:

Key	              Type	                 Description

top_spans	List[List[int]]	List of spaCy token indices pairs representing spans

document	List[str]	Document's tokens (from spaCy; but represented as string not Token)

clusters	List[List[List[int]]]	Clusters of spans (represented by token indices pairs)

In [4]:
# Input text
text = "Paul works at Google. He loves his job."

# Get coreference resolution
predictions = predictor.predict(document=text)

# Print the resolved text
resolved_text = predictions['document']
for cluster in predictions['clusters']:
    mention = cluster[0]
    referent = cluster[-1]
    resolved_text[mention[0]:mention[1]+1] = [resolved_text[referent[0]]]
    for idx in range(mention[0]+1, mention[1]+1):
        resolved_text[idx] = ""

print(' '.join(resolved_text))


He works at Google . He loves his job .


In [5]:
text = "Eva and Martha didn't want their friend Jenny to feel lonely so they invited her to the party in Las Vegas."
prediction = predictor.predict(document=text)

In [6]:
# it's our original text (with extra whitespaces as we trivialy just joined tokens with ' ')
' '.join(prediction['document'])

"Eva and Martha did n't want their friend Jenny to feel lonely so they invited her to the party in Las Vegas ."

In [7]:
# but that's how it looks after coreference resolution (notice the possessive!)
predictor.coref_resolved(text)

"Eva and Martha didn't want Eva and Martha's friend Jenny to feel lonely so Eva and Martha invited their friend Jenny to the party in Las Vegas."